## Tandem visualization and post-processing
Author: Jeena Yun (j4yun@ucsd.edu)

Did you successfully run Tandem? Now, we will learn about how to process Tandem outputs such that we can produce useful plots showing different aspects of rupture history.

### Learning objectives
- Learn about output formats of Tandem
- Learn about how to visualize outputs at different time and depth
- Learn about the effects of changing material properties on the rupture characteristics

### Contents

- [Understand fault probe outputs](#sec_understand_fp)
- [Prepare for plotting: load packages and define path to the outputs](#sec_load_packages)
- [Plot time series of a variable on an individual fault probe](#sec_plot_time_series)
- [Plot spatiotemporal evolution of slip rate](#sec_plot_slip_rate_image)
- [Compare two models](#sec_comp_models)
    - [Compare shear moduli ($\mu$)](#sec_comp_mu)
    - [Compare peak slip rates](#sec_comp_psr)
    - [Compare spatiotemporal evolution of slip rates](#sec_comp_srimage)
- [(Bonus) Plot spatiotemporal evolution of cumulative slip](#sec_bonus)

<a id='sec_understand_fp'></a>

## Understand fault probe outputs

Before we jump into plotting anything, let's learn about Tandem fault probe outputs.

In the parameter file, we provided **'name'** and **'location'** of on-fault probes. For example, in your `bp3.toml` file, you'll see:
```
[fault_probe_output]
prefix = "outputs/fltst_"
t_max = 9460800
probes = [
    { name = "dp000", x = [0.0, -0.0] },
    { name = "dp001", x = [0.9848077530122079, -0.17364817766693033] },
    { name = "dp002", x = [1.9696155060244158, -0.34729635533386066] },
    ...
]
```
As a result, you'll see files `fltst_dp000.csv`, `fltst_dp001.csv`, `fltst_dp002.csv`, ..., under `outputs` directory.

Each probe (or each csv file) will record the following variables (in 2D):
| Name               | Column Index | Description                                                            | Unit |
|--------------------|--------------|------------------------------------------------------------------------|------|
| **Time**           | 0            | Time at which the output is written                                    | s    |
| **state**          | 1            | State variable $\phi = f_0 + b \ln{\left(V_0 \theta / L\right)}$       | -    |
| **slip0**          | 2            | Fault slip                                                             | m    |
| **traction0**      | 3            | Shear traction on the fault                                            | MPa  |
| **slip-rate0**     | 4            | Slip rate on the fault                                                 | m/s  |
| **normal-stress**  | 5            | Normal stress on the fault                                             | MPa  |

Now, we know how the fault probe output looks like. Let's make some figures.

<a id='sec_load_packages'></a>

## Prepare for plotting: load packages and define path to the outputs

Let's first find where your output lives. All the job outputs you ran through the gateway are accessible through this **Jupyter Notebook Expanse** app.

To check the job names:

In [ ]:
!ls ../                 # putting ! in the front is equivalent to typing ls in terminal

Using the information shown above, define the path to your Tandem output.

In [ ]:
# save_dir = '../YOUR_JOB_NAME/'
save_dir = '/Users/j4yun/Desktop/quakeworx/tandem/training_test_BP3_highres/'

Then, load python packages that are commonly used throughout this notebook

In [ ]:
import numpy as np                       # contains a number of useful functions
import pandas as pd                      # efficiently loads csv files
import matplotlib.pyplot as plt          # essential for plotting 

yr2sec = 365*24*60*60               # A year in seconds; useful when plotting Tandem simulations of thousands of years
plt.rcParams['font.size'] = '15'    # Define default font size for plots

Now, we are ready to plot Tandem outputs!

<a id='sec_plot_time_series'></a>

## Plot time series of a variable on an individual fault probe

We will start from making the simplest plot by reading one of the csv files and plot a time series of a variable.

For example, let's make a time series of on-fault variables recorded at probe `fltst_dp044.csv`.

In [ ]:
# Define target csv file name
fname = save_dir + 'outputs/fltst_dp044.csv'

# Read csv file using pandas package
# Each field is seperated by commas, so use delimiter=','
# Skip the first row, which is a comment showing the probe location, to keep the header information
dat = pd.read_csv(fname, delimiter=',', skiprows=1)

All the variables are accessible using the header name:

In [ ]:
print(dat['slip0'])

It is useful to keep track of the probe location by reading the first row of the csv file:

In [ ]:
# Save station location by reading only the first row
probeloc_str = pd.read_csv(fname, nrows=1, header=None).values # the first row is saved as a string

# Extract x and y location of the probe from the string
probe_x,probe_y = float(probeloc_str[0][0].split('[')[-1]),float(probeloc_str[0][-1].split(']')[0])

Now, let's plot time series of slip rate and shear traction:

In [ ]:
# Plot time-series
fig, [ax,ax2] = plt.subplots(ncols=2, figsize = (14,6))

ax.plot(dat['Time']/yr2sec, np.log10(abs(dat['slip-rate0'])), color='k', lw=2.5)
ax.set_xlabel('Time [yr]', fontsize=17)
ax.set_ylabel('$\log_{10}$(Slip Rate [m/s])', fontsize=17)
ax.grid(True, alpha=0.5)

ax2.plot(dat['Time']/yr2sec, dat['traction0'], color='k', lw=2.5)
ax2.set_xlabel('Time [yr]', fontsize=17)
ax2.set_ylabel('Shear Traction [MPa]', fontsize=17)
ax2.grid(True, alpha=0.5)

fig.suptitle('Station: [%1.4f, %1.4f]'%(probe_x,probe_y), fontsize=20, fontweight = 'bold')
plt.tight_layout()
plt.show()

<a id='sec_plot_slip_rate_image'></a>

## Plot spatiotemporal evolution of slip rate

How about plotting the spatiotemporal evolution of variables along the entire fault? 

To do this, we need to save all outputs from all probes into a single array. The function below will read all fault probe outputs and save them into a numpy array.

In [ ]:
from glob import glob     # enables global search on fault probe outputs

def load_fault_probe_outputs(save_dir):
    # Search all fault probe outputs
    fnames = glob(save_dir + 'outputs/fltst_*.csv')

    print('Start computing output... ',end='')
    # Iteratively read all csv files and append data and station locations into an empty array 
    # The resulting array will have a dimension of (# probes, # time steps, # variables (=6))
    outputs,dep=[],[]
    for fname in np.sort(fnames):
        # Read and append data
        dat = pd.read_csv(fname, delimiter=',', skiprows=1)
        outputs.append(dat.values)

        # Read and append station depth information
        probeloc_str = pd.read_csv(fname, nrows=1, header=None)
        dep.append(float(probeloc_str.values[0][-1].split(']')[0]))

    # Convert to numpy array for convenience
    outputs = np.array(outputs)
    dep = np.array(dep)

    # Sort the array by depth and 
    ii = np.argsort(abs(dep))
    outputs = outputs[ii,:,:]
    dep = dep[ii]
    print('Done!')

    return outputs, dep

With this array, we can extract, e.g, slip rate, for all depths and time steps

In [ ]:
outputs,dep = load_fault_probe_outputs(save_dir)
sr = abs(outputs[:,:,4])       # slip rate is in 4th column

To plot it as a 2D image, we need a meshgrid of x- (time step) and y- (depth) coordinates:

In [ ]:
xax = np.arange(sr.shape[1])   # define x-axis as time steps
X,Y = np.meshgrid(xax,abs(dep))      # use meshgrid to combine x- and y- coordinates

Here, we import the colormap I specifically put together to make slip rate plots that better capture different rupture speeds:
- Red: coseismic slip rate
- Yellow: aseismic slip rate comparable to the reference slip rate (V0)
- Blue: stable creep of about plate loading rate (Vp)
- Black: locked

Like before, we are plotting the slip rate in log10 scale to see a wide spectrum of velocities.

In [ ]:
from cmap_for_sliprate import auto_cmap_for_sliprate
cmap, norm = auto_cmap_for_sliprate()

Using this colormap, we obtain a plot of spatiotemporal evolution of slip rate

In [ ]:
fig,ax=plt.subplots(figsize=(14,7)) # figure size
    
cb = plt.pcolormesh(X, Y, sr, cmap=cmap, norm=norm)    
plt.colorbar(cb,extend='both').set_label('Slip Rate [m/s]',fontsize=17,rotation=270,labelpad=30) # vertical colorbar

plt.xlabel('Timesteps',fontsize=17)
plt.ylabel('Depth [km]',fontsize=17)
plt.xlim(0,np.max(X))
plt.ylim(np.max(abs(Y)),0)

plt.tight_layout()

<a id='sec_comp_models'></a>

## Compare two models

A moment ago, we ran two models, one with a uniform shear modulus and the other with a depth-varying shear modulus $\mu$.

We will now learn how to compare material properties and resulting peak slip rate from different models.

First, define output path to each job.

In [ ]:
# save_dir = '../UNIFORM_JOBNAME/'
# save_dir_depvar = '../DEP_VAR_JOBNAME/'
save_dir = '/Users/j4yun/Desktop/quakeworx/tandem/training_test_BP3_highres/'
save_dir_depvar = '/Users/j4yun/Desktop/quakeworx/tandem/depth_varying_r/'

<a id='sec_comp_mu'></a>

### **Compare shear moduli ($\mu$)**

The functions defined in `.lua` file can be called in python using `LuaRuntime` from a package called `lupa`: 

In [ ]:
from lupa import LuaRuntime

# Define a function to call from Lua file
def load_from_lua(save_dir):
    # Initiate Lua callback
    lua = LuaRuntime()

    # Open and read Lua file
    with open(save_dir+'bp3.lua', "r") as f:    # you can change the name of lua file later
        lua_code = f.read()
    lua.execute(lua_code)                       # execute lines in Lua

    # 2. Access a specific scenario
    scenario = lua.globals().bp3_custom         # 'bp3_custom' is the name of the scenario we defined in the lua file
    return scenario

Now, load the scenairo information for each model:

In [ ]:
# Read lua files for both models
scenario = load_from_lua(save_dir)
scenario_depvar = load_from_lua(save_dir_depvar)

The variables and functions defined in each scenario can be executed: E.g.,

In [ ]:
print('amax =',scenario.amax) # constant 'amax' in the uniform mu model
print('mu(0,10) =',scenario.mu(scenario,0,10),'(uniform mu)') # function 'mu(x,y)' in the uniform mu model, executed at (x,y) = (0,10)
print('mu(0,10) =',scenario_depvar.mu(scenario_depvar,0,10),'(depth-varying mu)') # function 'mu(x,y)' in the depth-varying mu model, executed at (x,y) = (0,10)

Let's use this Lua callback to plot spatial distributions of $\mu$ in both models.

In [ ]:
# Set depth and distance ranges
depth = np.linspace(0, 50, 100)  # 100 points from 0 to 50
distance = np.linspace(-50, 50, 100)  # 100 points from -50 to 50

# Compute mu values in a double loop
mu = np.zeros((len(depth), len(distance)))            # empty array to save mu values (for uniform)
mu_depvar = np.zeros((len(depth), len(distance)))     # empty array to save mu values (for depth-varying)
for i, depth_i in enumerate(depth):
    for j, distance_j in enumerate(distance):
        mu[i, j] = scenario.mu(scenario, distance_j, depth_i)  # direct call into Lua
        mu_depvar[i, j] = scenario_depvar.mu(scenario_depvar, distance_j, depth_i)


# Plot results using imshow
fig, [ax,ax2] = plt.subplots(ncols=2, figsize=(14, 5))  # initiate a plot with two panels

# Panel 1. Unifrom mu
# We will use the same vmin & vmax for both panels for direct comparison
im = ax.imshow(mu, extent=[distance.min(), distance.max(), depth.max(), depth.min()], aspect='auto', vmin=np.min(mu_depvar), vmax=np.max(mu_depvar))
plt.colorbar(im,ax=ax,label=r'$\mu$ [GPa]')
ax.set_xlabel('Distance from Trench [km]', fontsize=17)
ax.set_ylabel('Depth [km]', fontsize=17)
ax.set_title(r'Uniform $\bf\mu$', fontsize=20, fontweight='bold')

# Panel 2. Depth-varying mu
im = ax2.imshow(mu_depvar, extent=[distance.min(), distance.max(), depth.max(), depth.min()], aspect='auto', vmin=np.min(mu_depvar), vmax=np.max(mu_depvar))
plt.colorbar(im,ax=ax2,label=r'$\mu$ [GPa]')
ax2.set_xlabel('Distance from Trench [km]', fontsize=17)
ax2.set_ylabel('Depth [km]', fontsize=17)
ax2.set_title(r'Depth-varying $\bf\mu$', fontsize=20, fontweight='bold')

plt.tight_layout()
plt.show()

<a id='sec_comp_psr'></a>

### **Compare peak slip rates**

We checked how to visualize differences in the material properties. Now, let's see how these differences in material properties affect the rupture dynamics.

Comparing peak slip rate is a good way to understand differences in slip rate and recurrence intervals.

Let's first compute the peak slip rate, using the array output. Array output for each model can be computed using the `load_fault_probe_outputs` function we defined above

In [ ]:
# Compute array outputs
outputs, dep = load_fault_probe_outputs(save_dir)
outputs_depvar, dep_depvar = load_fault_probe_outputs(save_dir_depvar)

# Extract slip rates
sr = outputs[:,:,4]
sr_depvar = outputs_depvar[:,:,4]

# Understand the size of the slip rate output
print(sr.shape)

Note the size of the slip rate output is (# probes, # time step). We want to compute the peak slip rate along depth for each time step. This can be done by using the `np.max` function with appropriate `axis` argument:

In [ ]:
# Compute peak slip rate along the entire fault
psr = np.max(sr,axis=0)
psr_depvar = np.max(sr_depvar,axis=0)

# Check the size of the peak slip rate output
print(psr.shape)

You will see that the peak slip rate output has a correct size equal to the number of time steps. Then, plot the two model results together to compare the difference.

In [ ]:
# Plot time-series
plt.figure(figsize = (10,6))
plt.plot(outputs[0,:,0]/yr2sec, np.log10(psr), color='k', lw=2.5, label=r'Uniform $\mu$') # outputs[0,:,0] contains time information at the surface
plt.plot(outputs_depvar[0,:,0]/yr2sec, np.log10(psr_depvar), color='r', lw=2.5, label=r'Depth-varying $\mu$')
plt.xlabel('Time [yr]', fontsize=17)
plt.ylabel('$log_{10}$(Peak Slip Rate [m/s])', fontsize=17)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.show()

<a id='sec_comp_srimage'></a>

### **Compare spatiotemporal evolution of slip rates**

Or, plot spatiotemporal variation of slip rate in the depth-varying $\mu$ model and compare it with the uniform $\mu$ model.

In [ ]:
# Define x- and y- coordinates
xax = np.arange(sr_depvar.shape[1])   # define x-axis as time steps
X,Y = np.meshgrid(xax,abs(dep_depvar))      # use meshgrid to combine x- and y- coordinates

# Define custom colormap
from cmap_for_sliprate import auto_cmap_for_sliprate
cmap, norm = auto_cmap_for_sliprate()

# Plot using pcolormesh
fig,ax=plt.subplots(figsize=(14,7)) # figure size
    
cb = plt.pcolormesh(X, Y, sr_depvar, cmap=cmap, norm=norm)    
plt.colorbar(cb,extend='both').set_label('Slip Rate [m/s]',fontsize=17,rotation=270,labelpad=30) # vertical colorbar

plt.xlabel('Timesteps',fontsize=17)
plt.ylabel('Depth [km]',fontsize=17)
plt.xlim(0,np.max(X))
plt.ylim(np.max(abs(Y)),0)

plt.tight_layout()

<a id='sec_bonus'></a>

## (Bonus) Plot spatiotemporal evolution of cumulative slip

Well done, all of you, who successfully followed the tutorial! You're now ready to visualize various aspects of your model. For those of you who are interested more in SEAS models, I'm leaving a small bonus scripts here.

Plotting spatiotemporal evolution of cumulative slip along the entire fault is very common practice to understand the rupture evolution of SEAS models. I provided a automatized script to create such plot.

The following script will plot coseismic slip evolution (in pink) and interseismic slip evolution (in grey). The slip evolution is expressed by slip contours, plotted every given `dt_coseismic` seconds (coseismic) and `dt_creep` years (interseismic).

In [ ]:
# Define input parameters
Vths = 1e-2                 # Slip rate threshold defining coseismic rupture [m/s]
dt_creep = 5                # Time interval for interseismic slip contour [yr]
dt_coseismic = 1            # Time interval for coseismic slip contour [s]

# Compute cumulative slip outputs
from cumslip_plot import compute_cumslip
cumslip_outputs = compute_cumslip(outputs,dep,Vths,dt_creep*yr2sec,dt_coseismic,print_on=True)

# Plot
fig,ax = plt.subplots(figsize = (8,6))
ax.plot(cumslip_outputs['cscoseis'], cumslip_outputs['depcoseis'], color=(200/255,110/255,110/255), lw=1)
ax.plot(cumslip_outputs['cscreep'], cumslip_outputs['depcreep'], color='0.62', lw=1)
ax.set_xlabel('Cumulative Slip [m]', fontsize=17)
ax.set_ylabel('Depth [km]', fontsize=17)
xl = ax.get_xlim()
ax.set_xlim(0,xl[1])
ax.set_ylim(max([np.max(np.absolute(cumslip_outputs['depcoseis'])),np.max(np.absolute(cumslip_outputs['depcreep']))]),0)
plt.tight_layout()
plt.show()